In [ ]:
twitter_neg = pd.read_pickle(path+'TWITTER_SEARCHES/MJI BIGRAMS (NEGATIVE)/twitter_music_magazine.pkl')
twitter_neg_2 = pd.read_pickle(path+'TWITTER_SEARCHES/MJI BIGRAMS (NEGATIVE)/twitter_music_oral_history.pkl')
twitter_neg['Target'] = '0'
twitter_neg_2['Target'] = '0'
twitter_neg = pd.concat([twitter_neg[['tweet', 'Target']], twitter_neg_2[['tweet', 'Target']]])
twitter_pos = pd.read_pickle(path+'TWITTER_SEARCHES/MUSOW BIGRAMS (POSITIVE)/twitter_music_archive.pkl')
twitter_pos['Target'] = '1'
twitter_pos = twitter_pos[['tweet', 'Target']]
twitter_pos

twitter_set = pd.concat([twitter_pos, twitter_neg])
twitter_set['Target'] = twitter_set['Target'].astype('int')
twitter_set = twitter_set.reset_index(drop=True)

#base set 
twitter_set_ce = twitter_set.copy()
# Specify the columns to encode then fit and transform
encoder_base = ce.BackwardDifferenceEncoder(cols=['tweet'])
twitter_set_ce = encoder_base.fit_transform(twitter_set_ce, verbose=1)
twitter_set_ce.drop(['intercept'], inplace=True, axis=1)